<a href="https://colab.research.google.com/github/LeeGitaek/2020_AI_Class/blob/master/soil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

토양 오염의 수치를 예측

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=7a25291164e27c7d4216172057bd72d8d40d8330ac3af002fe57a82ba52b11c7
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 29 13:26 kaggle.json


In [3]:
!kaggle competitions download -c 2020soil

  0% 0.00/27.2k [00:00<?, ?B/s]
100% 27.2k/27.2k [00:00<00:00, 22.0MB/s]


In [4]:
!unzip 2020soil.zip

Archive:  2020soil.zip
  inflating: 2020AI_soil_train.csv   
  inflating: 2020_soil_test.csv      
  inflating: soil_submission.csv     


In [5]:
import pandas as pd 
import numpy as np
import torch 
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [31]:
train = pd.read_csv('2020AI_soil_train.csv')
print(train.head(10))
print(train.info())


   Area    Cd    Cu    As    Hg    Pb     Zn    Ni  pollution level
0   1.0  2.13   9.4  2.82  0.01  14.6   58.0  10.9             1.60
1   2.0  1.83   9.3  7.29  0.06  17.2   36.8   9.6             3.03
2   3.0  0.90   8.3  6.90  0.02  13.4   31.4   9.8             1.69
3   4.0  0.00   1.5  2.03  0.00   4.2    7.9   2.7             0.00
4   5.0  1.57  14.6  3.39  0.00  30.6  127.4  11.8            11.04
5   6.0  0.40   0.0  0.00  0.00   9.3   30.3   3.2             0.00
6   7.0  1.87   9.8  1.89  0.01  21.6   59.8  12.9             1.34
7   8.0  2.40  10.2  4.44  0.00  20.2   61.5  10.8             1.87
8   9.0  1.40   4.3  0.00  0.00  19.6   50.2   3.1             0.87
9  10.0  0.60   5.2  0.00  0.00  17.8   35.4   5.1             0.07
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1269 entries, 0 to 1268
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             1269 non-null   float64
 1   Cd 

In [43]:
learning_rate = 0.001
training_epoch = 1000
batch_size = 50 

In [44]:
x_train = train.iloc[:,1:-1]
y_train = train.iloc[:,[-1]]

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

print(x_train.shape)
print(y_train.shape)

torch.Size([1269, 7])
torch.Size([1269, 1])


In [45]:
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)

data_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last=True)



layer 5개 , kaiming uniform 초기화

In [46]:
linear1 = nn.Linear(7,4,bias=True)
linear2 = nn.Linear(4,4,bias=True)
linear3 = nn.Linear(4,4,bias=True)
linear4 = nn.Linear(4,4,bias=True)
linear5 = nn.Linear(4,1,bias=True)

nn.init.xavier_uniform_(linear1.weight)
nn.init.kaiming_normal_(linear2.weight)
nn.init.xavier_uniform_(linear3.weight)
nn.init.kaiming_normal_(linear4.weight)
nn.init.xavier_uniform_(linear5.weight)
relu = nn.ReLU()

activation function - relu

In [47]:
model = nn.Sequential(
    linear1,relu,
    linear2,relu,
    linear3,relu,
    linear4,relu,
    linear5
).to(device)

In [48]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [49]:
total_batch = len(data_loader)

for epoch in range(training_epoch):
    avg_cost = 0
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('epoch {:.4f} , cost = {:.6f}'.format(epoch,avg_cost))
print('learning finished!')

epoch 0.0000 , cost = 623.259155
epoch 1.0000 , cost = 592.272949
epoch 2.0000 , cost = 549.943604
epoch 3.0000 , cost = 488.858521
epoch 4.0000 , cost = 427.558350
epoch 5.0000 , cost = 385.405823
epoch 6.0000 , cost = 366.797028
epoch 7.0000 , cost = 349.229248
epoch 8.0000 , cost = 346.884064
epoch 9.0000 , cost = 330.017426
epoch 10.0000 , cost = 326.075592
epoch 11.0000 , cost = 313.456909
epoch 12.0000 , cost = 312.616730
epoch 13.0000 , cost = 304.030670
epoch 14.0000 , cost = 296.079407
epoch 15.0000 , cost = 288.318878
epoch 16.0000 , cost = 280.941559
epoch 17.0000 , cost = 259.226135
epoch 18.0000 , cost = 264.415649
epoch 19.0000 , cost = 256.807281
epoch 20.0000 , cost = 249.539017
epoch 21.0000 , cost = 237.982498
epoch 22.0000 , cost = 233.695862
epoch 23.0000 , cost = 225.061371
epoch 24.0000 , cost = 218.552307
epoch 25.0000 , cost = 208.816193
epoch 26.0000 , cost = 203.035706
epoch 27.0000 , cost = 196.997040
epoch 28.0000 , cost = 188.551743
epoch 29.0000 , cost = 1

In [50]:
test = pd.read_csv('2020_soil_test.csv')
print(test.info())
test = test.iloc[:,1:]
test = np.array(test)
test = torch.FloatTensor(test).to(device)

with torch.no_grad():
    predict = model(test)
predict

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Area    227 non-null    int64  
 1   Cd      227 non-null    float64
 2   Cu      227 non-null    float64
 3   As      227 non-null    float64
 4   Hg      227 non-null    float64
 5   Pb      227 non-null    float64
 6   Zn      227 non-null    float64
 7   Ni      227 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 14.3 KB
None


tensor([[ 5.9477e+01],
        [ 4.8937e+01],
        [ 3.7951e-02],
        [ 8.0434e+00],
        [ 3.3670e+01],
        [-3.4261e-01],
        [ 1.1514e+00],
        [ 7.6254e+00],
        [ 6.7801e-01],
        [ 8.4418e-01],
        [-4.1892e-01],
        [ 7.6705e-01],
        [ 6.6318e-01],
        [ 9.9172e-01],
        [ 4.4758e+01],
        [ 6.7034e+01],
        [ 1.7025e+01],
        [ 7.5005e-02],
        [ 2.3971e+01],
        [ 4.5843e+01],
        [ 1.0961e+01],
        [ 6.6864e+01],
        [ 2.1810e+01],
        [ 5.0551e-01],
        [ 1.0626e+01],
        [ 9.4540e+00],
        [ 5.6097e+00],
        [ 3.0397e+01],
        [ 2.1901e+01],
        [ 5.1502e+00],
        [ 4.6209e+00],
        [ 2.9407e+01],
        [ 2.9469e+01],
        [ 6.9785e-01],
        [ 1.8370e+01],
        [ 6.3155e+00],
        [ 4.3948e+00],
        [ 7.9637e-01],
        [ 3.1602e-01],
        [ 4.7778e+00],
        [ 4.5420e+00],
        [ 1.1928e+00],
        [ 7.9513e-01],
        [ 6

In [51]:
correct_prediction = predict.cpu().numpy().reshape(-1,1)
result = pd.read_csv('soil_submission.csv')

In [52]:
for i in range(len(correct_prediction)):
    result['Expected'][i] = correct_prediction[i]
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [53]:
result.to_csv('submit.csv',index=False)

In [54]:
!kaggle competitions submit -c 2020soil -f submit.csv -m "14010974_이기택_기말고사파트2-3"

100% 4.92k/4.92k [00:07<00:00, 696B/s]
Successfully submitted to 2020ai_soil